<a href="https://colab.research.google.com/github/jmmoralesf/TFM/blob/master/RELUELUDROPINCRELU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cd '/content/gdrive/My Drive/tensor'

/content/gdrive/My Drive/tensor


In [0]:
!pip install tensorflow-gpu==2.0
!pip install tensorboard

In [0]:
from __future__ import print_function, division

import argparse
import os
import random
import time
import sys
import nibabel as nib
import numpy as np
import statsmodels.api as sm
import tensorflow as tf
from tensorflow.keras import backend
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, concatenate, AveragePooling2D, MaxPooling2D,Dropout
from tensorflow.keras.optimizers import Adam
from scipy import ndimage
from scipy.signal import argrelextrema
print ("139")

backend.set_image_data_format = 'channels_last'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


def pad_image(vol, padsize):
    dim = vol.shape
    padsize = np.asarray(padsize, dtype=int)
    dim2 = dim + 2 * padsize
    temp = np.zeros(dim2, dtype=np.float16)
    temp[padsize:dim[0] + padsize, padsize:dim[1] + padsize, padsize:dim[2] + padsize] = vol
    return temp


def normalize_image(vol, contrast):

    temp = vol[np.nonzero(vol)].astype(float)

    q = np.percentile(temp, 99)
    temp = temp[temp <= q]
    temp = temp.reshape(-1, 1)
    bw = q / 80
    print("99th quantile is %.4f, gridsize = %.4f" % (q, bw))
    
    kde = sm.nonparametric.KDEUnivariate(temp)
    
    kde.fit(kernel='gau', bw=bw, gridsize=80, fft=True)
    x_mat = 100.0 * kde.density
    y_mat = kde.support
    
    indx = argrelextrema(x_mat, np.greater)
    indx = np.asarray(indx, dtype=int)
    heights = x_mat[indx][0]
    peaks = y_mat[indx][0]
    peak = 0.00
    print("%d peaks found." % (len(peaks)))
    
    # norm_vol = vol
    if contrast.lower() == "t1":
        peak = peaks[-1]
        print("Peak found at %.4f for %s" % (peak, contrast))
        # norm_vol = vol/peak
        # norm_vol[norm_vol > 1.25] = 1.25
        # norm_vol = norm_vol/1.25
    elif contrast.lower() in ['t2', 'pd', 'fl']:
        peak_height = np.amax(heights)
        idx = np.where(heights == peak_height)
        peak = peaks[idx]
        print("Peak found at %.4f for %s" % (peak, contrast))
        # norm_vol = vol / peak
        # norm_vol[norm_vol > 3.5] = 3.5
        # norm_vol = norm_vol / 3.5
    else:
        print("Contrast must be either T1,T2,PD, or FL. You entered %s. Returning 0." % contrast)
    
    # return peak, norm_vol
    return peak


def get_patches(invol1, invol2, mask, patchsize):
    rng = random.SystemRandom()
    dsize = np.floor(patchsize / 2).astype(np.int)
    
    indx = np.array(np.nonzero(mask))
    num_patches = len(indx[0])
    
    print("Number of patches used  = %d " % num_patches)
    randindx = rng.sample(range(num_patches), num_patches)
    newindx = np.ndarray((3, num_patches), dtype=np.int)
    for i in range(num_patches):
        for j in range(3):
            newindx[j, i] = indx[j, randindx[i]]
    
    matsize = (num_patches, patchsize[0], patchsize[1], 1)
    
    blurmask = ndimage.filters.gaussian_filter(mask.astype(np.float32), sigma=(1, 1, 1))
    blurmask[blurmask < 0.0001] = 0
    blurmask = blurmask * 100  # Just to have reasonable looking error values during training, otherwise
    # the error values become too small
    
    t1_patches = np.zeros(matsize, dtype=np.float32)
    fl_patches = np.zeros(matsize, dtype=np.float32)
    mask_patches = np.zeros(matsize, dtype=np.float32)
    
    for i in range(0, num_patches):
        x = newindx[0, i]
        y = newindx[1, i]
        z = newindx[2, i]
        
        t1_patches[i, :, :, 0] = invol1[x - dsize[0]:x + dsize[0] + 1, y - dsize[1]:y + dsize[1] + 1, z]
        fl_patches[i, :, :, 0] = invol2[x - dsize[0]:x + dsize[0] + 1, y - dsize[1]:y + dsize[1] + 1, z]
        mask_patches[i, :, :, 0] = blurmask[x - dsize[0]:x + dsize[0] + 1, y - dsize[1]:y + dsize[1] + 1, z]
    
    return t1_patches, fl_patches, mask_patches




139


In [0]:

def get_inception_2d(inlayer, base_filters):
    conv_inception1a = Conv2D(base_filters * 4, (1, 1), activation='relu', padding='same')(inlayer)
    
    conv_inception2a = Conv2D(base_filters * 6, (1, 1), activation='relu', padding='same')(inlayer)
    conv_inception4a = Conv2D(base_filters * 8, (3, 3), activation='relu', padding='same')(conv_inception2a)
    
    conv_inception3a = Conv2D(base_filters, (1, 1), activation='relu', padding='same')(inlayer)
    conv_inception5a = Conv2D(base_filters * 2, (5, 5), activation='relu', padding='same')(conv_inception3a)
    
    pool_inception1a = AveragePooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(inlayer)
    conv_inception6a = Conv2D(base_filters * 2, (1, 1), activation='relu', padding='same')(pool_inception1a)
    
    pool_inception2a = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(inlayer)
    conv_inception7a = Conv2D(base_filters * 2, (1, 1), activation='relu', padding='same')(pool_inception2a)
    
    outlayer = concatenate([conv_inception1a, conv_inception4a], axis=-1)
    outlayer = concatenate([outlayer, conv_inception5a], axis=-1)
    outlayer = concatenate([outlayer, conv_inception6a], axis=-1)
    outlayer = concatenate([outlayer, conv_inception7a], axis=-1)
    return outlayer
def get_inception(inlayer, base_filters):
    tower_1 = Conv2D(64, (1,1), padding='same', activation='relu')(inlayer)
    tower_1 = Conv2D(64, (3,3), padding='same', activation='relu')(tower_1)
    tower_2 = Conv2D(64, (1,1), padding='same', activation='relu')(inlayer)
    tower_2 = Conv2D(64, (5,5), padding='same', activation='relu')(tower_2)
    tower_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(inlayer)
    tower_3 = Conv2D(64, (1,1), padding='same', activation='relu')(tower_3)
    output =concatenate([tower_1, tower_2, tower_3], axis = 3)
    return output



def get_model():
    ds = 2
    #b = lambda x: tf.keras.layers.LeakyReLU(alpha=0.1)(x)
    b='relu'
    b1='elu'
     #b = lambda x: tf.keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1, 2])(x)
    #b1=lambda x: tf.keras.layers.PReLU(alpha_initializer='zeros',alpha_regularizer=None, alpha_constraint=None)(x) 
    #, alpha_regularizer=None, alpha_constraint=None, shared_axes=None [1, 2]
    metricas = [  tf.keras.metrics.Accuracy(name='accuracy')]
    
    t1 = tf.keras.Input((None, None, 1))
    conv1a = Conv2D(128, (3, 3), activation=b, padding='same')(t1)
    conv1a2=Dropout(0.1)(conv1a)
    conv2a = Conv2D(128 // ds, (5, 5), activation=b, padding='same')(conv1a2)
    conv2a2=Dropout(0.1)(conv2a)
    conv3a = Conv2D(128 // (ds * 2), (3, 3), activation=b, padding='same')(conv2a2)
    conv3a2=Dropout(0.1)(conv3a)
    conv4a = Conv2D(128 // (ds ** 3), (5, 5), activation=b, padding='same')(conv3a2)
    conv4a2=Dropout(0.1)(conv4a)
    conv5a = Conv2D(128 // (ds ** 4), (3, 3), activation=b, padding='same')(conv4a2)
    conv6a=Dropout(0.1)(conv5a)
   
    
    fl = tf.keras.Input((None, None, 1))
    conv1b = Conv2D(128, (3, 3), activation=b, padding='same')(fl)
    conv1b2=Dropout(0.1)(conv1b)
    conv2b = Conv2D(128 // ds, (5, 5), activation=b, padding='same')(conv1b2)
    conv2b2=Dropout(0.1)(conv2b)
    conv3b = Conv2D(128 // (ds * 2), (3, 3), activation=b, padding='same')(conv2b2)
    conv3b2=Dropout(0.1)(conv3b)
    conv4b = Conv2D(128 // (ds ** 3), (5, 5), activation=b, padding='same')(conv3b2)
    conv4b2=Dropout(0.1)(conv4b)
    conv5b = Conv2D(128 // (ds ** 4), (3, 3), activation=b, padding='same')(conv4b2)
    conv6b=Dropout(0.1)(conv5b)
    
    
    ####Añadimos inception modules##

    inception3 = get_inception(conv6a, base_filters)
    #inception2 = get_inception(inception1, base_filters)
    #inception3 = get_inception(inception2, base_filters)

    inception6 = get_inception(conv6b, base_filters)
    #inception5 = get_inception(inception1, base_filters)
    #inception6 = get_inception(inception2, base_filters)

    concat = concatenate([inception3, inception6], axis=-1)
    
    conv1c = Conv2D(128, (3, 3), activation=b1, padding='same')(concat)
    conv2c = Conv2D(128 // ds, (5, 5), activation=b1, padding='same')(conv1c)
    conv3c = Conv2D(128 // (ds * 2), (3, 3), activation=b1, padding='same')(conv2c)
    conv4c = Conv2D(128 // (ds ** 3), (5, 5), activation=b1, padding='same')(conv3c)
    conv5c = Conv2D(128 // (ds ** 4), (3, 3), activation=b1, padding='same')(conv4c)
    
    conv_last = Conv2D(1, (3, 3), activation=b1, padding='same')(conv5c)
    
    model = Model(inputs=[t1, fl], outputs=conv_last)
    model.compile(optimizer=Adam(lr=0.0001), loss='mean_squared_error',metrics = ['accuracy'] )
    
    return model

In [0]:

checkpoint_path = '/content/gdrive/My Drive/tensor/relueludropincrelu/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
base_filters=3
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True,verbose=1)
                                                
                                                 
reduce_lr = tf.compat.v2.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.2,patience=5, min_lr=0.001)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
#def main(atlas_dir, numatlas, patchsize, out_dir):
   
    #results = parser.parse_args()
out_dir = os.path.abspath('/content/gdrive/My Drive/tensor/modelos')
    
atlas_dir = os.path.abspath('/content/gdrive/My Drive/tensor')
    
print("Keras parameters are as follows -- ")
print("Backend           : " + backend.backend())
print("Float             : " + backend.floatx())
print("Image Data Format : " + backend.image_data_format())
    
if backend.floatx() != 'float32':
    print("WARNING: Data type should be float32 to save on memory.")
##### ATLAS EPOCH PSIZE #####   
natlas=139
epoc=15
psize='35x35'





#    elif results.gpu is not None:
        # Change gpu id to run on different gpu
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
print("Using GPU id " + str(os.environ["CUDA_VISIBLE_DEVICES"]))
    
print('Atlas Directory     =', atlas_dir)
print('Number of atlases   =', natlas)
print('Number o f epochs  =',epoc)
print('Patch size          =', psize)
print('Output Directory    =', out_dir)
    
psize = [int(item) for item in psize.split('x')]
    
print('Patch size          = %d x %d ' % (psize[0], psize[1]))
patchsize=psize   
   
batchsize = 128
patchsize = np.array(patchsize)
padsize = np.max(patchsize + 1) / 2
num_patches = 0
for i in range(natlas):
    num_patches += int(nib.load(os.path.join(atlas_dir, "atlas" + str(i + 1) + "_mask.nii.gz")).get_data().sum())
    
print("Total number of lesion patches = " + str(num_patches))
ident = time.strftime("%d-%m-%Y") + "_" + time.strftime("%H-%M-%S")
print("Unique ID is %s " % ident)
    
patch_str = str(int(patchsize[0])) + "x" + str(int(patchsize[1]))





#### Nombre de GRABADO###
outname = os.path.join(out_dir, "relueludropincrelu139_"+ patch_str + ".h5")







print("Trained model will be written at %s" % outname)

patsize = (num_patches, patchsize[0], patchsize[1], 1)
matsize = (num_patches, patchsize[0], patchsize[1], 1)
t1_patches = np.zeros(matsize, dtype=np.float32)
fl_patches = np.zeros(matsize, dtype=np.float32)
mask_patches = np.zeros(matsize, dtype=np.float32)
    
count2 = 0
count1 = 0
for i in range(0, natlas):
    t1name = os.path.join(atlas_dir, "atlas" + str(i + 1) + "_" + "T1.nii.gz")
    print("Reading %s" % t1name)
    t1 = nib.load(t1name).get_data().astype(np.float32)

        
    flname = os.path.join(atlas_dir, "atlas" + str(i + 1) + "_" + "FL.nii.gz")
    print("Reading %s" % flname)
    fl = nib.load(flname).get_data().astype(np.float32)
        
    maskname = os.path.join(atlas_dir, "atlas" + str(i + 1) + "_" + "mask.nii.gz")
    print("Reading %s" % maskname)
    mask = nib.load(maskname).get_data().astype(np.float32)
        
    # Normalize the images
    t1 = np.array(t1 / normalize_image(t1, 'T1'), dtype=np.float32)
    fl = np.array(fl / normalize_image(fl, 'FL'), dtype=np.float32)
        
    dim = t1.shape
    print("Image size = %d x %d x %d " % (dim[0], dim[1], dim[2]))
        
    padded_t1 = pad_image(t1, padsize)
    padded_fl = pad_image(fl, padsize)
    padded_mask = pad_image(mask, padsize)
        
    t1_patches_a, fl_patches_a, mask_patches_a = get_patches(padded_t1, padded_fl, padded_mask, patchsize)
        
    dim = t1_patches_a.shape
    count2 = count1 + dim[0]
    print("Atlas %d : indices [%d,%d]" % (i + 1, count1, count2 - 1))
    t1_patches[count1:count2, :, :, :] = t1_patches_a
    fl_patches[count1:count2, :, :, :] = fl_patches_a
    mask_patches[count1:count2, :, :, :] = mask_patches_a
    count1 = count1 + dim[0]

t1_patches = np.asarray(t1_patches,dtype=np.float16)
fl_patches = np.asarray(fl_patches, dtype=np.float16)
mask_patches = np.asarray(mask_patches, dtype=np.float16)
    
print("Total number of patches collected = " + str(count2))
    
print("Size of the input matrix is " + str(mask_patches.shape))
model = get_model()
model.summary()
model.fit([t1_patches, fl_patches], mask_patches, batch_size=batchsize, epochs=epoc, verbose=1, validation_split=0.2,callbacks=[cp_callback,callback,reduce_lr])
print("Model is written at " + outname)
model.save(outname)

model.metrics.count('loss') 
#results = model.evaluate([t1_patches, fl_patches], mask_patches,  batch_size=batchsize)
#print('test loss, test acc:', results)

Keras parameters are as follows -- 
Backend           : tensorflow
Float             : float32
Image Data Format : channels_last
Using GPU id 0
Atlas Directory     = /content/gdrive/My Drive/tensor
Number of atlases   = 139
Number o f epochs  = 15
Patch size          = 35x35
Output Directory    = /content/gdrive/My Drive/tensor/modelos
Patch size          = 35 x 35 
Total number of lesion patches = 888057
Unique ID is 16-12-2019_15-40-39 
Trained model will be written at /content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5
Reading /content/gdrive/My Drive/tensor/atlas1_T1.nii.gz
Reading /content/gdrive/My Drive/tensor/atlas1_FL.nii.gz
Reading /content/gdrive/My Drive/tensor/atlas1_mask.nii.gz
99th quantile is 348.6189, gridsize = 4.3577
4 peaks found.
Peak found at 320.4581 for T1
99th quantile is 158.9887, gridsize = 1.9874
5 peaks found.
Peak found at 104.6066 for FL
Image size = 240 x 256 x 256 
Number of patches used  = 2473 
Atlas 1 : indices [0,2472]
Reading /co

0

In [0]:

%cd '/content/gdrive/My Drive/tensor'

/content/gdrive/My Drive/tensor


CALCULO DE PREDICCIONES

In [0]:
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas160_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas160_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas159_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas159_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas158_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas158_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas157_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas157_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas156_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas156_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas155_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas155_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas154_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas154_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas153_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas153_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas152_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas152_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas151_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas151_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas150_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas150_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas149_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas149_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas148_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas148_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas147_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas147_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas146_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas146_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas145_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas145_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas144_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas144_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas143_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas143_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas142_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas142_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas141_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas141_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0
!python FTest.py --models '/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5' --t1 '/content/gdrive/My Drive/tensor/dicei/atlas140_T1.nii.gz' --fl '/content/gdrive/My Drive/tensor/dicei/atlas140_FL.nii.gz' --o '/content/gdrive/My Drive/tensor/relueludropincrelu139/' --gpu 0














Instructions for updating:
non-resource variables are not supported in the long term
1
Using GPU id 0
1 models found at
/content/gdrive/My Drive/tensor/modelos/relueludropincrelu139_35x35.h5
Temporary directory :/tmp/tmpeix0ofq_
T1 image         = /content/gdrive/My Drive/tensor/dicei/atlas160_T1.nii.gz
FLAIR image      = /content/gdrive/My Drive/tensor/dicei/atlas160_FL.nii.gz
Output directory = /content/gdrive/My Drive/tensor/relueludropincrelu139
99th quantile is 204.8200, gridsize = 2.5602
2 peaks found.
Peak found at 183.1120 for t1
99th quantile is 91.7762, gridsize = 1.1472
2 peaks found.
Peak found at 65.8805 for fl
Predicting memberships.
bucle t in range
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Time taken for 1st model = 15.37 seconds
Writing /content/gdrive/My Drive/tensor/relueludropincrelu139/atlas160_T1_LesionMembership.nii.gz
Thresholding memberships at 0.34 and removing 18-connected objects with volume <27 voxels.
Writing /content

CALCULO DE DICE


In [0]:
!pip install nipype
from nipype.algorithms import metrics
overlap = metrics.Overlap()


     |████████████████████████████████| 3.2MB 4.9MB/s 
     |████████████████████████████████| 430kB 51.7MB/s 
     |████████████████████████████████| 92kB 11.1MB/s 
     |████████████████████████████████| 235kB 44.5MB/s 
     |████████████████████████████████| 430kB 53.9MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 348kB 48.0MB/s 
  Created wheel for traits: filename=traits-5.2.0-cp36-cp36m-linux_x86_64.whl size=353239 sha256=b4047f33d189c10695f10fd272a623d7434f0aad4300ea8c4db57bd6cd5e58ed
  Stored in directory: /root/.cache/pip/wheels/e5/a5/8e/77963012fd6e6b37b6e9a6b13222dad51df159417abf8bdf3d
  Created wheel for simplejson: filename=simplejson-3.17.0-cp36-cp36m-linux_x86_64.whl size=114208 sha256=a5272ed9fb657df8d8e6f5df1db2ac9bfd74e9d09a13951d8e1513b140e53482
  Stored in directory: /root/.cache/pip/wheels/86/c0/83/dcd0339abb2640544bb8e0938aab2d069cef55e5647ce6e097
Successfully built traits simplejson


In [0]:

overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas160_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas160_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)

overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas159_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas159_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)
overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas158_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas158_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)
overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas157_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas157_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)
overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas156_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas156_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)
overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas155_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas155_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)
overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas154_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas154_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)

overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas153_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas153_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)
overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas152_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas152_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)
overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas151_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas151_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)
overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas150_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas150_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)





overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas149_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas149_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)
overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas148_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas148_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)
overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas147_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas147_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)
overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas146_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas146_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)
overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas145_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas145_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)
overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas144_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas144_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)

overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas143_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas143_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)
overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas142_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas142_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)
overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas141_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas141_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)
overlap.inputs.volume1 = '/content/gdrive/My Drive/tensor/dicei/atlas140_mask.nii.gz'
overlap.inputs.volume2 = '/content/gdrive/My Drive/tensor/relueludropincrelu139/atlas140_T1_LesionMask.nii.gz'
res = overlap.run()
print(res.outputs.dice)


0.5187
0.31659
0.63583
0.59277
0.50558
0.42395
0.65266
0.35734
0.54949
0.66911
0.53642
0.2126
0.64588
0.24976
0.33706
0.33261
0.62023
0.4919
0.49558
0.63688
0.42588
